In [1]:
# !pip install googletrans==3.1.0a0 # for the translation 
# !pip install farm-haystack # for the QuestionGenerator

### - Loading the Data

In [8]:
import json
import datetime

now = datetime.datetime.now()
# File path
file_path = r"C:\A\00Master\Demo\data acc\att\output\DIN 1045-101" # Replace with the  file path

# Initialize an empty list to hold the data
data = []

# Read the JSONL file line by line
with open(file_path, 'r') as file:
    for line in file:
        # Decode each line as a separate JSON object
        json_object = json.loads(line)
        data.append(json_object)

# Now, 'data' is a list of dictionaries loaded from each line of the JSONL file
import os
def extract_file_name_without_extension(file_path):
    # Extract the file name from the file path
    file_name_with_extension = os.path.basename(file_path)
    # Split the file name and the extension
    file_name_without_extension, _ = os.path.splitext(file_name_with_extension)
    return file_name_without_extension

filename = extract_file_name_without_extension(file_path)
filename

'DIN 1045-101'

### -   prepare the data to the model

In [9]:


# Initialize an empty list to store items without 'table' in any key
non_table_items = []

# Iterate over each item in the data
for item in data:
    # Check if the item is a dictionary
    if isinstance(item, dict):
        # Assume the item does not have 'table' in keys, until proven otherwise
        has_table = False

        # Iterate over each key in the dictionary
        for key in item.keys():
            # Check if 'table' is in the key
            if 'table' in key:
                # If yes, set has_table to True and break the loop
                has_table = True
                break

        # If the item does not have 'table' in any key, add it to the list
        if not has_table:
            non_table_items.append(item)

# non_table_items now contains all items without 'table' in their keys
print(non_table_items)
data=non_table_items

all_values = []

# Iterate through each dictionary in the list
for d in data:
    # Iterate through each key-value pair in the dictionary
    for key, value in d.items():
        # Add the value to the list
        all_values.append(value)
all_values

[{'DIN 1045-101_page_0-0': 'ICS Bemessung Konstruktion Stahlbeton Spannbetontragwerken Teil Konformitätsnachweis Design of concrete structures Part Evaluation of Conformity for brick floors in accordance with Calcul des structures en ton Partie Evaluation de la conformite pour plancher en briques conforme Gesamtumfang Seiten DINNormenausschuss Bauwesen NABau © Deutsches Institut Normung Jede Artder Vervielfältigung auch auszugsweise Berlin gestattet une IIINIIHINIININN'}, {'DIN 1045-101_page_2-0': 'Vorwort Dieses Dokument wurde den Arbeitsausschüssen NA AA „Bemessung Konstruktion“ NA AA „Mauerwerksbau“ des DINNormenausschusses Bauwesen NABau erarbeitet Es wird auf Möglichkeit hingewiesen dass einige Elemente dieses Dokuments Patentrechte berühren können ist nicht dafür verantwortlich einige oder alle diesbezüglichen Patentrechte identifizieren Die Normenreihe Tragwerke aus Beton Stahlbeton Spannbeton besteht aus DIN1045 Tragwerke aus Beton Stahlbeton Spannbeton— Teil2 Beton Festlegung 

### - 	Question And Answer Generation (QAG)

In [15]:
from haystack.nodes import QuestionGenerator
from haystack.nodes import QuestionGenerator
from googletrans import Translator, LANGUAGES
import datetime

now = datetime.datetime.now()

# Initialize the QuestionGenerator
qg = QuestionGenerator()
translator = Translator()



# Dictionary to store  results: question as key and text as value
questions_to_text_dict = {}

# Combine English and German question starters
question_starters = [
    "what", "how", "why", "who", "where", "when", "which", "whose", "are", "can",
    "could", "do", "does", "did", "have", "has", "is", "should", "will", "would",
    # Add German question starters
    "was", "wie", "warum", "wer", "wo", "wann", "welcher", "welche", "welches", "wessen",
    "sind", "kann", "könnte", "tun", "macht", "gemacht", "haben", "hat", "ist", "sollte",
    "wird", "würde", "woher", "weshalb", "wieso", "wozu", "in welche", "in welchem",
    "in welcher", "in welchen", "in welches", "aus welchem grund", "an wen", "mit wem",
    "zu wem", "ab wann", "bis wann", "aus welcher", "mit welcher", "zu welcher"
]
for text in all_values:
            
        generated_questions = qg.generate(text)
        for question in generated_questions:
                if question.lower().startswith(tuple(question_starters)):
                    translated_question = translator.translate(question, dest='de').text
                
                # Store the text as the value for the translated question key
                    questions_to_text_dict[translated_question] = text


translated_questions_dict = {}

# Translate each key (question) to German
for key, value in questions_to_text_dict.items():
    translated_key = translator.translate(key, src='en', dest='de').text
    translated_questions_dict[translated_key] = value

# Print the translated questions and their corresponding texts
for question, text in translated_questions_dict.items():
    print(f"Question: {question}, Text: {text}")
    
stop = datetime.datetime.now()
haystack = stop - now
print(haystack)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Question: Was macht Nabau Deutsches Institut Normung Jede Artder Vervielfältigung Auch Ausugsweise Berlin Gestattet une iiiniihiniininn?, Text: ICS Bemessung Konstruktion Stahlbeton Spannbetontragwerken Teil Konformitätsnachweis Design of concrete structures Part Evaluation of Conformity for brick floors in accordance with Calcul des structures en ton Partie Evaluation de la conformite pour plancher en briques conforme Gesamtumfang Seiten DINNormenausschuss Bauwesen NABau © Deutsches Institut Normung Jede Artder Vervielfältigung auch auszugsweise Berlin gestattet une IIINIIHINIININN
Question: Wofür steht na aa?, Text: Vorwort Dieses Dokument wurde den Arbeitsausschüssen NA AA „Bemessung Konstruktion“ NA AA „Mauerwerksbau“ des DINNormenausschusses Bauwesen NABau erarbeitet Es wird auf Möglichkeit hingewiesen dass einige Elemente dieses Dokuments Patentrechte berühren können ist nicht dafür verantwortlich einige oder alle diesbezüglichen Patentrechte identifizieren Die Normenreihe Tragwe